In [3]:
! pip install deepctr

     |████████████████████████████████| 122kB 7.0MB/s 


In [20]:
import warnings
warnings.filterwarnings("ignore")
import itertools
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import namedtuple
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  MinMaxScaler, LabelEncoder
from deepctr.feature_column import SparseFeat, DenseFeat, VarLenSparseFeat

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [21]:
data = pd.read_csv('/content/gdrive/My Drive/dl_group/modtrain.csv', sep="\t")
data

,Unnamed: 0,size,model_attr,user_attr,user_id,brand,category,rating,movie_id,hist_movie_id,hist_len
0,0,4.0,0,0,0,10,2,0,12,0 0 0 0 0 0 0 0 0 0 0,1
1,1,4.0,0,0,0,13,1,0,44,0 65 0 0 0 0 0 0 0 0 0,2
2,2,4.0,0,0,0,10,0,0,49,0 65 1 0 0 0 0 0 0 0 0,3
3,3,4.0,0,0,0,10,1,0,71,0 65 1 5 0 0 0 0 0 0 0,4
4,4,4.0,0,0,0,13,1,0,16,0 65 1 5 13 0 0 0 0 0 0,5
...,...,...,...,...,...,...,...,...,...,...,...
68593,68593,4.0,0,0,7621,10,2,0,56,43 12 48 52 23 0 0 0 0 0 0,5
68594,68594,4.0,0,0,7621,4,0,0,74,43 12 48 52 23 87 0 0 0 0 0,6
68595,68595,4.0,0,0,7621,1,0,0,25,43 12 48 52 23 87 55 0 0 0 0,7
68596,68596,4.0,0,0,7621,12,2,0,82,43 12 48 52 23 87 55 24 0 0 0,8


In [22]:
data=data[['user_attr', 'user_id', 'brand','category', 'rating', 'movie_id', 'hist_movie_id', 'hist_len']]
data.columns = ['gender', 'user_id', 'age','movie_type_id', 'label', 'movie_id', 'hist_movie_id', 'hist_len']
X = data.drop('label', axis=1)
y = data['label']

In [23]:
X_train = {"user_id": np.array(X["user_id"]), \
            "gender": np.array(X["gender"]), \
            "age": np.array(X["age"]), \
            "hist_movie_id": np.array([[int(i) for i in l.split(' ')] for l in X["hist_movie_id"]]), \
            "hist_len": np.array(X["hist_len"]), \
            "movie_id": np.array(X["movie_id"]), \
            "movie_type_id": np.array(X["movie_type_id"])}
y_train = np.array(y)

In [24]:
from collections import namedtuple

SparseFeat = namedtuple('SparseFeat', ['name', 'vocabulary_size', 'embedding_dim'])
DenseFeat = namedtuple('DenseFeat', ['name', 'dimension'])
VarLenSparseFeat = namedtuple('VarLenSparseFeat', ['name', 'vocabulary_size', 'embedding_dim', 'maxlen'])

feature_columns = [SparseFeat('user_id', max(data["user_id"])+1, embedding_dim=8),
                    SparseFeat('gender', max(data["gender"])+1, embedding_dim=8),
                    SparseFeat('age', max(data["age"])+1, embedding_dim=8),
                    SparseFeat('movie_id', max(data["movie_id"])+1, embedding_dim=8),
                    SparseFeat('movie_type_id', max(data["movie_type_id"])+1, embedding_dim=8),
                    DenseFeat('hist_len', 1)]
feature_columns += [VarLenSparseFeat('hist_movie_id', vocabulary_size=max(data["movie_id"])+1, embedding_dim=8, maxlen=50)]


behavior_feature_list = ['movie_id']
behavior_seq_feature_list = ['hist_movie_id']


In [1]:
# input
def build_input_layers(feature_columns):
    input_layer_dict = {}

    for fc in feature_columns:
        if isinstance(fc, SparseFeat):
            input_layer_dict[fc.name] = Input(shape=(1,), name=fc.name)
        elif isinstance(fc, DenseFeat):
            input_layer_dict[fc.name] = Input(shape=(fc.dimension,), name=fc.name)
        elif isinstance(fc, VarLenSparseFeat):
            input_layer_dict[fc.name] = Input(shape=(fc.maxlen,), name=fc.name)

    return input_layer_dict


# embedding
def build_embedding_layers(feature_columns, input_layer_dict):
    embedding_layer_dict = {}

    for fc in feature_columns:
        if isinstance(fc, SparseFeat):
            embedding_layer_dict[fc.name] = Embedding(fc.vocabulary_size, fc.embedding_dim, name='emb_' + fc.name)
        elif isinstance(fc, VarLenSparseFeat):
            embedding_layer_dict[fc.name] = Embedding(fc.vocabulary_size + 1, fc.embedding_dim, name='emb_' + fc.name, mask_zero=True)

    return embedding_layer_dict


def embedding_lookup(feature_columns, input_layer_dict, embedding_layer_dict):
    embedding_list = []

    for fc in feature_columns:
        _input = input_layer_dict[fc]
        _embed = embedding_layer_dict[fc]
        embed = _embed(_input)
        embedding_list.append(embed)

    return embedding_list


In [26]:
# Dice
class Dice(Layer):
    def __init__(self):
        super(Dice, self).__init__()
        self.bn = BatchNormalization(center=False, scale=False)

    def build(self, input_shape):
        self.alpha = self.add_weight(shape=(input_shape[-1],), dtype=tf.float32, name='alpha')

    def call(self, x):
        x_normed = self.bn(x)
        x_p = tf.sigmoid(x_normed)

        return self.alpha * (1.0 - x_p) * x + x_p * x

In [27]:
# ActivationUnit
class LocalActivationUnit(Layer):
    def __init__(self, hidden_units=(256, 128, 64), activation='prelu'):
        super(LocalActivationUnit, self).__init__()
        self.hidden_units = hidden_units
        self.linear = Dense(1)
        self.dnn = [Dense(unit, activation=PReLU() if activation == 'prelu' else Dice()) for unit in self.hidden_units]

    def call(self, inputs):
        # query: item Embedding，keys: hist Embedding
        query, keys = inputs
        keys_len = keys.get_shape()[1]
        queries = tf.tile(query, multiples=[1, keys_len, 1])
        att_input = tf.concat([queries, keys, queries - keys, queries * keys], axis=-1)
        att_out = att_input
        for fc in self.dnn:
            att_out = fc(att_out)
        att_out = self.linear(att_out)  # B x len x 1
        att_out = tf.squeeze(att_out, -1)  # B x len

        return att_out


class AttentionPooling(Layer):
    def __init__(self, att_hidden_units=(128, 64)):
        super(AttentionPooling, self).__init__()
        self.att_hidden_units = att_hidden_units
        self.local_att = LocalActivationUnit(self.att_hidden_units, 'dice')

    def call(self, inputs):
        query, keys = inputs
        key_masks = tf.not_equal(keys[:, :, 0], 0)
        attention_score = self.local_att([query, keys])
        outputs = attention_score
        paddings = tf.zeros_like(attention_score)
        outputs = tf.where(key_masks, attention_score, paddings)
        outputs = tf.expand_dims(outputs, axis=1)  # B x 1 x len
        # keys : B x len x emb_dim
        outputs = tf.matmul(outputs, keys)  # B x 1 x emb_dim
        outputs = tf.squeeze(outputs, axis=1)  # B x emb_dim

        return outputs

In [28]:

def get_dnn_logits(dnn_input, hidden_units=(200, 80), activation='prelu'):
    dnns = [Dense(unit, activation=PReLU() if activation == 'prelu' else Dice()) for unit in hidden_units]

    dnn_out = dnn_input
    for dnn in dnns:
        dnn_out = dnn(dnn_out)
    dnn_logits = Dense(1, activation='sigmoid')(dnn_out)
    return dnn_logits


def concat_embedding_list(feature_columns, input_layer_dict, embedding_layer_dict, flatten=False):
    embedding_list = []
    for fc in feature_columns:
        _input = input_layer_dict[fc.name]  
        _embed = embedding_layer_dict[fc.name]  # B x 1 x dim  
        embed = _embed(_input)  # B x dim  
        if flatten:
            embed = Flatten()(embed)

        embedding_list.append(embed)

    return embedding_list

def concat_input_list(input_list):
    feature_nums = len(input_list)
    if feature_nums > 1:
        return Concatenate(axis=1)(input_list)
    elif feature_nums == 1:
        return input_list[0]
    else:
        return None


def DIN(feature_columns, behavior_feature_list, behavior_seq_feature_list):
    input_layer_dict = build_input_layers(feature_columns)
    input_layers = list(input_layer_dict.values())
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeat), feature_columns))
    dense_feature_columns = list(filter(lambda x: isinstance(x, DenseFeat), feature_columns))
    dnn_dense_input = []
    for fc in dense_feature_columns:
        dnn_dense_input.append(input_layer_dict[fc.name])
   
    dnn_dense_input = concat_input_list(dnn_dense_input)
    embedding_layer_dict = build_embedding_layers(feature_columns, input_layer_dict)

    
    dnn_sparse_emb_input = concat_embedding_list(sparse_feature_columns, input_layer_dict, embedding_layer_dict,
                                                 flatten=True)
    dnn_sparse_input = concat_input_list(dnn_sparse_emb_input)

   
    query_emb_list = embedding_lookup(behavior_feature_list, input_layer_dict, embedding_layer_dict)
    keys_emb_list = embedding_lookup(behavior_seq_feature_list, input_layer_dict, embedding_layer_dict)

    dnn_seq_input_list = []
    for i in range(len(keys_emb_list)):
        seq_emb = AttentionPooling()([query_emb_list[i], keys_emb_list[i]])
        dnn_seq_input_list.append(seq_emb)
    dnn_seq_input = concat_input_list(dnn_seq_input_list)

    
    dnn_input = Concatenate(axis=1)([dnn_dense_input, dnn_sparse_input, dnn_seq_input])
    dnn_logits = get_dnn_logits(dnn_input, activation='dice')
    model = Model(input_layers, dnn_logits)
    return model




In [ ]:
model = DIN(feature_columns, behavior_feature_list, behavior_seq_feature_list)
model.summary()
learning_rate = 1e-4
adam = tf.optimizers.Adam(lr=learning_rate)
model.compile(optimizer=adam,
              loss="mse",
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error'), tf.keras.metrics.MeanSquaredError(name='mean_squared_error')])
history = model.fit(X_train, y_train, batch_size=12, epochs=5, validation_split=0.2)

In [ ]:
# plot
def training_vis(hist):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    mse = hist.history['mean_squared_error']
    val_mse = hist.history['val_mean_squared_error']

    rme = hist.history['root_mean_squared_error']
    val_rme = hist.history['val_root_mean_squared_error']

    # make a figure
    fig = plt.figure(figsize=(12, 4))
    # subplot loss
    ax1 = fig.add_subplot(131)
    ax1.plot(loss, label='train_loss')
    ax1.plot(val_loss, label='val_loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss on Training and Validation Data')
    ax1.legend()
    # subplot auc
    ax2 = fig.add_subplot(132)
    ax2.plot(mse, label='train_mse')
    ax2.plot(val_mse, label='val_mse')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('MSE')
    ax2.set_title('MSE  on Training and Validation Data')
    ax2.legend()
    plt.tight_layout()

    # subplot binary_crossentropy
    ax3 = fig.add_subplot(133)
    ax3.plot(rme, label='train_root_mean_squared_error')
    ax3.plot(val_rme, label='val_root_mean_squared_error')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('root_mean_squared_error')
    ax3.set_title('RMSE on Training and Validation Data')
    ax3.legend()
    plt.tight_layout()

    plt.show()

training_vis(history)